In [1]:
import pyaudio
import wave
import whisper

# Функция для записи аудио
def record_audio(filename="command.wav", duration=5):
    chunk = 1024
    sample_format = pyaudio.paInt16
    channels = 1
    fs = 44100
    frames = []

    p = pyaudio.PyAudio()
    
    print("Recording...")
    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=fs,
                    frames_per_buffer=chunk,
                    input=True)

    for _ in range(0, int(fs / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)
    
    print("Recording finished")

    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(sample_format))
    wf.setframerate(fs)
    wf.writeframes(b''.join(frames))
    wf.close()

# Функция для распознавания речи
def transcribe_audio(filename="command.wav"):
    model = whisper.load_model("base")
    result = model.transcribe(filename)
    return result['text']


In [2]:
import nltk
import pymorphy3

nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

morph = pymorphy3.MorphAnalyzer()
stop_words = set(stopwords.words('russian'))

def preprocess_text(text):
    # Токенизация
    tokens = word_tokenize(text.lower())
    # Удаление стоп-слов
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    # Лемматизация
    lemmas = [morph.parse(token)[0].normal_form for token in tokens]
    return lemmas


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ziyat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ziyat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import subprocess
import webbrowser

COMMANDS = {
    'поиск': 'search',
    'видео': 'video',
    'открыть': 'open'
}

def execute_command(lemmas):
    if 'поиск' in lemmas:
        query = ' '.join(lemmas[lemmas.index('поиск')+1:])
        url = f"https://yandex.ru/search/?text={query}"
        webbrowser.open(url)
    elif 'видео' in lemmas:
        query = ' '.join(lemmas[lemmas.index('видео')+1:])
        url = f"https://www.youtube.com/results?search_query={query}"
        webbrowser.open(url)
    elif 'телеграм' in lemmas:
        subprocess.run(["telegram-desktop"])
    elif 'дискорд' in lemmas:
        subprocess.run(["discord"])
    else:
        print("Команда не распознана")


In [7]:
record_audio()
text = transcribe_audio()
print(f"Распознанная команда: {text}")

lemmas = preprocess_text(text)
print(f"Леммы: {lemmas}")

execute_command(lemmas)

Recording...
Recording finished
Распознанная команда:  поиск кошка
Леммы: ['поиск', 'кошка']
